In [25]:
import joblib
import function_library as fl
import importlib
import pandas as pd
import time
importlib.reload(fl)

<module 'function_library' from 'C:\\Users\\leere\\PycharmProjects\\Football_ML3\\function_library.py'>

In [26]:
file_path = r"C:\Users\leere\OneDrive\Desktop\RAW DATA\GOALS_LIVE.CSV"

# List of common encodings to try
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

data = None
for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding)
        print(f"Successfully read the file with encoding: {encoding}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {encoding}")

col_dict = {
    "country": "Country",
    "league": "League",
    "datameci": "Date",
    "etapa": "Round",
    "txtechipa1": "home_team",
    "txtechipa2": "away_team",
    "place1t": "Home_team_place_total",
    "place1a": "Home_team_place_home",
    "place2t": "Away_team_place_total",
    "place2d": "Away_team_place_away",
    "customh": "ELO_home",
    "customa": "ELO_away",
    "custom3": "FORM_home",
    "custom4": "FORM_away",
    "home_val": "home_win",
    "home_val_2": "home_win_15",
    "home_val_3": "home_o25",
    "home_val_4": "home_o35",
    "home_val_5": "home_scored",
    "away_val": "away_win",
    "away_val_2": "away_win_15",
    "away_val_3": "away_o25",
    "away_val_4": "away_o35",
    "away_val_5": "away_scored",
    "scor1": "home_goals",
    "scor2": "away_goals",
    "cotao": "o2.5_odds",
}

data = data.rename(columns=col_dict).filter(items=col_dict.values())
# Convert Date column to datetime type
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Order by date
data = data.sort_values(by='Date')

data = data[data["Round"] >= 8]
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

cols_to_drop = ['Date','home_team', 'away_team', 'home_goals', 'away_goals', 'o2.5_odds']

data_ready =data.drop(columns=cols_to_drop)
data_ready

Failed to decode with encoding: utf-8
Successfully read the file with encoding: latin1


,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,ELO_home,ELO_away,FORM_home,...,home_win,home_win_15,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored
0,Poland,Ekstraklasa,22,10,11,8,6,1860.0,1931.0,-10.0,...,40.000,10.000,30.000,20.000,1.300,40.000,10.000,60.000,40.000,1.700
1,Bulgaria,Premier L,22,11,13,12,13,1871.0,1944.0,5.0,...,20.000,0.000,40.000,30.000,1.000,20.000,10.000,30.000,10.000,0.500
2,Denmark,Superligaen,19,8,8,6,6,1829.0,1862.0,-4.0,...,50.000,12.500,62.500,50.000,2.000,25.000,25.000,50.000,37.500,1.500
3,Netherlands,Eerste Divisie,21,8,10,10,2,1412.0,1431.0,4.0,...,46.154,15.385,53.846,15.385,1.538,58.333,33.333,58.333,33.333,1.667
4,Netherlands,Eerste Divisie,27,16,12,5,1,1395.0,1508.0,3.0,...,41.667,33.333,50.000,25.000,1.417,69.231,30.769,46.154,46.154,2.077
5,Netherlands,Eerste Divisie,27,19,20,3,4,1270.0,1583.0,-21.0,...,16.667,8.333,91.667,75.000,1.750,53.846,7.692,53.846,30.769,1.308
6,Spain,Primera,25,13,14,7,7,1705.0,1671.0,0.0,...,41.667,0.000,25.000,8.333,0.917,45.455,0.000,45.455,18.182,0.909
7,Portugal,Primeira L,23,4,2,7,10,2041.0,1878.0,-2.0,...,81.818,63.636,54.545,27.273,2.455,18.182,9.091,27.273,27.273,0.909
8,Romania,Liga 1,28,11,13,8,10,1920.0,1936.0,2.0,...,23.077,15.385,30.769,23.077,1.154,23.077,7.692,76.923,30.769,1.385
9,Turkey,Super Lig,25,1,2,2,2,2389.0,2312.0,-3.0,...,81.818,18.182,81.818,54.545,2.545,66.667,50.000,58.333,58.333,2.667


In [27]:
# Directory where models are saved
models_dir = "ALL_MODELS"

# Group unseen data by League and Sub-League
grouped_data = data_ready.groupby(['Country', 'League'])

# Dictionary to store predictions
predictions = {}

# Iterate through each group in the unseen data
for (country, league), group in grouped_data:
    # Construct the model filename
    model_filename = f"{models_dir}/{country}_{league}_model.pkl"

    try:
        # Load the corresponding model
        saved_data = joblib.load(model_filename)  # Load the entire dictionary
        model = saved_data['model']  # Extract the model
        scaler = saved_data['scaler']  # Extract the scaler
        threshold = saved_data['threshold']  # Extract the threshold

        # Extract features for prediction (exclude grouping columns)
        X = group.drop(columns=['Country', 'League'])

        # Scale the features
        X_scaled = scaler.transform(X)

        # Make probability predictions using the model
        y_proba = model.predict_proba(X_scaled)[:, 1]  # Positive class probabilities

        # Filter predictions based on the threshold
        group['Probability'] = y_proba
        group['Prediction'] = (y_proba >= threshold).astype(int)

        # Keep only rows where Prediction == 1
        filtered_predictions = group[group['Prediction'] == 1]

        # Add `home_team` and `away_team` from the original `data`
        game_info = data.loc[filtered_predictions.index, ['home_team', 'away_team', 'Date']]
        filtered_predictions = pd.concat([game_info, filtered_predictions], axis=1)

        # Add filtered predictions to the dictionary
        predictions[(country, league)] = filtered_predictions
        print(f"Filtered predictions made for {country} - {league}")

    except FileNotFoundError:
        print(f"Model not found for {country} - {league}. Skipping...")

    except Exception as e:
        print(f"Error processing {country} - {league}: {e}")

# Combine all predictions into a single DataFrame
if predictions:
    all_predictions = pd.concat(predictions.values(), ignore_index=True)
    print("\nAll Predictions as a DataFrame:")
    print(all_predictions)
else:
    print("No predictions available.")


Filtered predictions made for Austria - Erste Liga
Filtered predictions made for Bulgaria - Premier L
Filtered predictions made for Denmark - Superligaen
Filtered predictions made for England - Championship
Filtered predictions made for England - League One
Filtered predictions made for England - League Two
Filtered predictions made for England - Premier L
Filtered predictions made for Israel - Premier League
Filtered predictions made for Italy - Serie A
Filtered predictions made for Netherlands - Eerste Divisie
Filtered predictions made for Poland - Ekstraklasa
Filtered predictions made for Portugal - Primeira L
Filtered predictions made for Romania - Liga 1
Filtered predictions made for Scotland - Championship
Filtered predictions made for Scotland - Premier L
Filtered predictions made for Slovakia - Superliga
Filtered predictions made for Spain - Primera
Filtered predictions made for Turkey - Super Lig

All Predictions as a DataFrame:
          home_team     away_team       Date    

In [28]:
#all_predictions

In [29]:
all_predictions = fl.team_name_map(all_predictions)
# Get today's date (normalized to midnight so we only compare dates)

In [30]:
today = pd.Timestamp.today().normalize()
# Filter the DataFrame to include only rows with dates greater than or equal to today
all_predictions = all_predictions[all_predictions['Date'] >= today]
all_predictions

,home_team,away_team,Date,Country,League,Round,Home_team_place_total,Home_team_place_home,Away_team_place_total,Away_team_place_away,...,home_o25,home_o35,home_scored,away_win,away_win_15,away_o25,away_o35,away_scored,Probability,Prediction
0,ASK Voitsberg,Bregenz,2025-02-25,Austria,Erste Liga,17,12,14,4,12,...,50.000,0.000,0.750,28.571,14.286,42.857,14.286,1.143,0.618014,1
1,Septemvri,Krumovgrad,2025-02-24,Bulgaria,Premier L,22,11,13,12,13,...,40.000,30.000,1.000,20.000,10.000,30.000,10.000,0.500,1.000000,1
2,Wigan,Huddersfield,2025-02-25,England,League One,24,16,23,5,4,...,40.000,6.667,0.867,53.333,26.667,60.000,33.333,1.733,0.660185,1
3,Gillingham,Fleetwood Town,2025-02-25,England,League Two,26,19,19,14,13,...,46.667,13.333,1.133,26.667,20.000,53.333,40.000,1.400,0.536678,1
4,Tranmere,Accrington,2025-02-25,England,League Two,22,22,20,21,18,...,25.000,12.500,0.813,25.000,12.500,62.500,25.000,1.188,0.560959,1
5,Wolves,Fulham,2025-02-25,England,Premier L,27,17,17,10,9,...,66.667,25.000,1.333,41.667,16.667,41.667,25.000,1.500,0.589664,1
6,Brighton,Bournemouth,2025-02-25,England,Premier L,27,9,12,5,5,...,58.333,33.333,1.417,46.154,30.769,76.923,53.846,2.077,0.621825,1
7,Nottingham,Arsenal,2025-02-26,England,Premier L,27,3,5,2,2,...,41.667,25.000,1.833,53.846,38.462,30.769,30.769,1.846,0.623214,1
8,Jong PSV Eindhoven,FC Dordrecht,2025-02-24,Netherlands,Eerste Divisie,27,19,20,3,4,...,91.667,75.000,1.750,53.846,7.692,53.846,30.769,1.308,0.878788,1
9,LKP Motor Lublin,GKS Katowice,2025-02-24,Poland,Ekstraklasa,22,10,11,8,6,...,30.000,20.000,1.300,40.000,10.000,60.000,40.000,1.700,0.636364,1


In [31]:
fl.create_import_file(all_predictions, r"C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv", provider="lay_all_u25", market_name="Over/Under 2.5 Goals", selection_name="Under 2.5 Goals")

File created and saved successfully at: C:\Users\leere\OneDrive\Desktop\IMPORTS\LAY_U25_ALL_IMPORT.csv
